In [ ]:
!pip install pyspark

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=5eb6d2b3165564277a38be809608556f539854f30174260adf6ef4d82d5db5f4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [45]:
# Import the column_listings_df Parquet file
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("ReadParquet").getOrCreate()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the Parquet file
listings_df = spark.read.parquet('/content/drive/MyDrive/column_listings_df.parquet')
listings_df.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+------------------+--------------------+--------------------+---------+--------------------+---------------+----------+-----------------+-------------------+-------------------------+----------------------+----------------------------+------------------+------------------+---------------+------------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------+-----------------+---------------------+----------------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+------------------------------+-----------------+----+-----------+---------------------+-------+----------------+---+----+----------+-------+-------+------------+----------+---------------------+---------+-------

In [46]:
print(listings_df.columns)

['listing_id', 'listing_url', 'name', 'host_id', 'host_url', 'host_name', 'host_since', 'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month', 'Wifi', 'Smoke alarm', 'Carbon monoxide alarm', 'Kitchen', 'Air conditioning', 'TV', 'Iron', 'Essentials', 'Hangers', 'Shampoo', 'Refrigerator', 'Hair dryer', 'Dishes and silverware', 'Hot water', 'Cooking basics', 'Heating', 'Bed linens', 'Microwave', 'Oven', 'Fire ex

In [47]:
# Select only the desired columns
cleanup_df = listings_df[['room_type','accommodates','bathrooms_text','bedrooms','beds','neighbourhood_group_cleansed','price','Wifi','Smoke alarm',
                          'Carbon monoxide alarm','Kitchen','Air conditioning','TV','Iron','Essentials','Hangers','Shampoo','Refrigerator','Hair dryer',
                          'Dishes and silverware','Hot water','Cooking basics','Heating','Bed linens','Microwave','Oven','Fire extinguisher',
                          'Coffee maker','Free street parking','First aid kit','Self check-in','Dedicated workspace']]

#Rename columns
cleanup_df = cleanup_df.withColumnRenamed('neighbourhood_group_cleansed', 'neighbourhood_group')

cleanup_df.show()

+---------------+------------+--------------+--------+----+-------------------+-------+----+-----------+---------------------+-------+----------------+---+----+----------+-------+-------+------------+----------+---------------------+---------+--------------+-------+----------+---------+----+-----------------+------------+-------------------+-------------+-------------+-------------------+
|      room_type|accommodates|bathrooms_text|bedrooms|beds|neighbourhood_group|  price|Wifi|Smoke alarm|Carbon monoxide alarm|Kitchen|Air conditioning| TV|Iron|Essentials|Hangers|Shampoo|Refrigerator|Hair dryer|Dishes and silverware|Hot water|Cooking basics|Heating|Bed linens|Microwave|Oven|Fire extinguisher|Coffee maker|Free street parking|First aid kit|Self check-in|Dedicated workspace|
+---------------+------------+--------------+--------+----+-------------------+-------+----+-----------+---------------------+-------+----------------+---+----+----------+-------+-------+------------+----------+-----

In [48]:
cleanup_df.printSchema()

root
 |-- room_type: string (nullable = true)
 |-- accommodates: string (nullable = true)
 |-- bathrooms_text: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- price: string (nullable = true)
 |-- Wifi: integer (nullable = true)
 |-- Smoke alarm: integer (nullable = true)
 |-- Carbon monoxide alarm: integer (nullable = true)
 |-- Kitchen: integer (nullable = true)
 |-- Air conditioning: integer (nullable = true)
 |-- TV: integer (nullable = true)
 |-- Iron: integer (nullable = true)
 |-- Essentials: integer (nullable = true)
 |-- Hangers: integer (nullable = true)
 |-- Shampoo: integer (nullable = true)
 |-- Refrigerator: integer (nullable = true)
 |-- Hair dryer: integer (nullable = true)
 |-- Dishes and silverware: integer (nullable = true)
 |-- Hot water: integer (nullable = true)
 |-- Cooking basics: integer (nullable = true)
 |-- Heating: integer (nullable = true)
 |-- Bed l

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Fill missing values in 'bedrooms' with 1
cleanup_df = cleanup_df.withColumn('bedrooms', col('bedrooms').cast('int')).fillna(1, subset=['bedrooms'])

# Remove the dollar sign and convert to float
cleanup_df = cleanup_df.withColumn('price', cleanup_df['price'].substr(2, 100).cast('float'))

# Convert string columns to integer
cleanup_df = cleanup_df.withColumn('accommodates', col('accommodates').cast('integer'))
cleanup_df = cleanup_df.withColumn('beds', col('beds').cast('integer'))

cleanup_df.printSchema()

root
 |-- room_type: string (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- bathrooms_text: string (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- beds: integer (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- price: float (nullable = true)
 |-- Wifi: integer (nullable = true)
 |-- Smoke alarm: integer (nullable = true)
 |-- Carbon monoxide alarm: integer (nullable = true)
 |-- Kitchen: integer (nullable = true)
 |-- Air conditioning: integer (nullable = true)
 |-- TV: integer (nullable = true)
 |-- Iron: integer (nullable = true)
 |-- Essentials: integer (nullable = true)
 |-- Hangers: integer (nullable = true)
 |-- Shampoo: integer (nullable = true)
 |-- Refrigerator: integer (nullable = true)
 |-- Hair dryer: integer (nullable = true)
 |-- Dishes and silverware: integer (nullable = true)
 |-- Hot water: integer (nullable = true)
 |-- Cooking basics: integer (nullable = true)
 |-- Heating: integer (nullable = true)
 |-- Bed

In [50]:
from pyspark.sql.functions import split, regexp_extract, when

# Extract the number of bathrooms
cleanup_df = cleanup_df.withColumn('bathrooms', regexp_extract('bathrooms_text', r'(\d+(\.\d+)?)', 1).cast('float'))

# Extract the bathroom type
cleanup_df = cleanup_df.withColumn('bathroom_type',
                          when(cleanup_df['bathrooms_text'].contains('private'), 'private')
                          .when(cleanup_df['bathrooms_text'].contains('shared'), 'shared')
                          .otherwise('private')
                         )

# Reorder columns
cleanup_df = cleanup_df.select('room_type','accommodates','bedrooms','beds','bathrooms','bathroom_type','neighbourhood_group','price','Wifi','Smoke alarm',
                          'Carbon monoxide alarm','Kitchen','Air conditioning','TV','Iron','Essentials','Hangers','Shampoo','Refrigerator','Hair dryer',
                          'Dishes and silverware','Hot water','Cooking basics','Heating','Bed linens','Microwave','Oven','Fire extinguisher',
                          'Coffee maker','Free street parking','First aid kit','Self check-in','Dedicated workspace')

# Drop the 'bathrooms_text' column
cleanup_df = cleanup_df.drop('bathrooms_text')

cleanup_df.show()

+---------------+------------+--------+----+---------+-------------+-------------------+-----+----+-----------+---------------------+-------+----------------+---+----+----------+-------+-------+------------+----------+---------------------+---------+--------------+-------+----------+---------+----+-----------------+------------+-------------------+-------------+-------------+-------------------+
|      room_type|accommodates|bedrooms|beds|bathrooms|bathroom_type|neighbourhood_group|price|Wifi|Smoke alarm|Carbon monoxide alarm|Kitchen|Air conditioning| TV|Iron|Essentials|Hangers|Shampoo|Refrigerator|Hair dryer|Dishes and silverware|Hot water|Cooking basics|Heating|Bed linens|Microwave|Oven|Fire extinguisher|Coffee maker|Free street parking|First aid kit|Self check-in|Dedicated workspace|
+---------------+------------+--------+----+---------+-------------+-------------------+-----+----+-----------+---------------------+-------+----------------+---+----+----------+-------+-------+--------

In [51]:
pandas_df = cleanup_df.toPandas()

In [52]:
pandas_df.head()

,room_type,accommodates,bedrooms,beds,bathrooms,bathroom_type,neighbourhood_group,price,Wifi,Smoke alarm,...,Heating,Bed linens,Microwave,Oven,Fire extinguisher,Coffee maker,Free street parking,First aid kit,Self check-in,Dedicated workspace
0,Entire home/apt,4,1,2.0,1.0,private,Queens,171.0,1,1,...,0,1,1,0,0,1,1,0,1,0
1,Entire home/apt,7,3,4.0,1.0,private,Queens,311.0,1,1,...,0,1,1,0,0,0,0,1,1,0
2,Entire home/apt,12,3,7.0,1.0,private,Queens,328.0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,Entire home/apt,4,1,2.0,1.0,private,Queens,139.0,1,1,...,0,1,0,0,0,1,0,0,0,0
4,Entire home/apt,16,5,11.0,2.5,private,Queens,485.0,1,1,...,0,1,0,0,1,0,0,0,1,0


In [58]:
# Clean Pandas dataframe
pandas_df.dropna()
pandas_df['bedrooms']= pandas_df['bedrooms'].fillna(1)
pandas_df['bathrooms']= pandas_df['bathrooms'].fillna(1)
pandas_df = pandas_df.dropna(subset=['beds'])
pandas_df = pandas_df.dropna(subset=['price'])
pandas_df = pandas_df[~pandas_df['beds'].isin([float('inf')])]
pandas_df['beds'] = pandas_df['beds'].astype('int')

columns_to_rename = ['Wifi','Smoke alarm','Carbon monoxide alarm','Kitchen','Air conditioning','TV','Iron','Essentials','Hangers','Shampoo','Refrigerator','Hair dryer',
                'Dishes and silverware','Hot water','Cooking basics','Heating','Bed linens','Microwave','Oven','Fire extinguisher',
                'Coffee maker','Free street parking','First aid kit','Self check-in','Dedicated workspace']

# Convert amenities columns to lowercase and replace spaces with underscores
for column in columns_to_rename:
    pandas_df = pandas_df.rename(columns={column: column.lower().replace(' ', '_')})

pandas_df = pandas_df.rename(columns={'self_check-in': 'self_check_in'})

print(pandas_df.columns)

Index(['room_type', 'accommodates', 'bedrooms', 'beds', 'bathrooms',
       'bathroom_type', 'neighbourhood_group', 'price', 'wifi', 'smoke_alarm',
       'carbon_monoxide_alarm', 'kitchen', 'air_conditioning', 'tv', 'iron',
       'essentials', 'hangers', 'shampoo', 'refrigerator', 'hair_dryer',
       'dishes_and_silverware', 'hot_water', 'cooking_basics', 'heating',
       'bed_linens', 'microwave', 'oven', 'fire_extinguisher', 'coffee_maker',
       'free_street_parking', 'first_aid_kit', 'self_check_in',
       'dedicated_workspace'],
      dtype='object')


In [59]:
# Encode data
encoded_df = pd.get_dummies(pandas_df, columns=['room_type', 'bathroom_type', 'neighbourhood_group'], prefix=['room_type', 'bathroom_type', 'neighbourhood'])
encoded_df.columns

Index(['accommodates', 'bedrooms', 'beds', 'bathrooms', 'price', 'wifi',
       'smoke_alarm', 'carbon_monoxide_alarm', 'kitchen', 'air_conditioning',
       'tv', 'iron', 'essentials', 'hangers', 'shampoo', 'refrigerator',
       'hair_dryer', 'dishes_and_silverware', 'hot_water', 'cooking_basics',
       'heating', 'bed_linens', 'microwave', 'oven', 'fire_extinguisher',
       'coffee_maker', 'free_street_parking', 'first_aid_kit', 'self_check_in',
       'dedicated_workspace', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'bathroom_type_private',
       'bathroom_type_shared', 'neighbourhood_Brooklyn',
       'neighbourhood_Manhattan', 'neighbourhood_Queens'],
      dtype='object')

In [60]:
# Reorder columns
encoded_df = encoded_df[['price', 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room', 'accommodates', 'bedrooms',
                         'beds', 'bathrooms', 'bathroom_type_private', 'bathroom_type_shared', 'neighbourhood_Brooklyn', 'neighbourhood_Manhattan', 'neighbourhood_Queens',
                         'wifi', 'smoke_alarm', 'carbon_monoxide_alarm', 'kitchen', 'air_conditioning', 'tv', 'iron','essentials', 'hangers', 'shampoo', 'refrigerator',
                         'hair_dryer', 'dishes_and_silverware', 'hot_water', 'cooking_basics', 'heating', 'bed_linens', 'microwave', 'oven', 'fire_extinguisher', 'coffee_maker',
                         'free_street_parking', 'first_aid_kit', 'self_check_in', 'dedicated_workspace']]

# Drop any rows that have null values - will not work in linear regression model
encoded_df.dropna()
model_df = encoded_df.copy()

In [61]:
# # Export to csv here to use in VSCode
model_df.to_csv('/content/drive/MyDrive/model_df.csv')

# Linear Regression

In [ ]:
y = model_df["price"]
X = model_df.drop(columns="price")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [62]:
y_pred_regr = regr.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_regr = mean_squared_error(y_test, y_pred_regr)

# Calculate Root Mean Squared Error (RMSE)
rmse_regr = np.sqrt(mse_regr)

# Calculate R-squared for the deep learning model
r2_regr = r2_score(y_test, y_pred_regr)

print("Linear Regression Model - Root Mean Squared Error (RMSE):", rmse_regr)
print("Linear Regression Model - R-squared:", r2_regr)

Linear Regression Model - Root Mean Squared Error (RMSE): 101.29308277475644
Linear Regression Model - R-squared: 0.4483761872370835


In [63]:
# Score the model
print(f"Training Data Score: {regr.score(X_train, y_train)}")
print(f"Testing Data Score: {regr.score(X_test, y_test)}")

Training Data Score: 0.4378408904224498
Testing Data Score: 0.4483761872370835


# Deep Learning

In [ ]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=38, activation="relu", input_dim=38))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1))

In [ ]:
nn.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
fit_model = nn.fit(X_train, y_train, epochs=100, validation_split=0.25)

Epoch 1/100
476/476 [==============================] - 2s 5ms/step - loss: 8721.8057 - mean_absolute_error: 58.3794 - val_loss: 9191.0264 - val_mean_absolute_error: 60.6151
Epoch 2/100
476/476 [==============================] - 2s 5ms/step - loss: 8680.8975 - mean_absolute_error: 58.2244 - val_loss: 8874.7012 - val_mean_absolute_error: 58.7036
Epoch 3/100
476/476 [==============================] - 2s 4ms/step - loss: 8610.9092 - mean_absolute_error: 57.9843 - val_loss: 8778.3379 - val_mean_absolute_error: 60.0653
Epoch 4/100
476/476 [==============================] - 2s 3ms/step - loss: 8579.5742 - mean_absolute_error: 58.0238 - val_loss: 8864.3164 - val_mean_absolute_error: 57.8346
Epoch 5/100
476/476 [==============================] - 1s 3ms/step - loss: 8558.4863 - mean_absolute_error: 57.8587 - val_loss: 9023.9248 - val_mean_absolute_error: 57.4622
Epoch 6/100
476/476 [==============================] - 1s 3ms/step - loss: 8493.4268 - mean_absolute_error: 57.5409 - val_loss: 9028.46

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

212/212 [==============================] - 0s 2ms/step - loss: 9829.9463 - mean_absolute_error: 61.9395
Loss: 9829.9462890625, Accuracy: 61.93950271606445


In [ ]:
# Calculate predictions using the deep learning model
y_pred_nn = nn.predict(X_test)

# Calculate Mean Squared Error (MSE) for the deep learning model
mse_nn = mean_squared_error(y_test, y_pred_nn)

# Calculate Root Mean Squared Error (RMSE) for the deep learning model
rmse_nn = np.sqrt(mse_nn)

# Calculate R-squared for the deep learning model
r2_nn = r2_score(y_test, y_pred_nn)

print("Deep Learning Model - Root Mean Squared Error (RMSE):", rmse_nn)
print("Deep Learning Model - R-squared:", r2_nn)

212/212 [==============================] - 1s 2ms/step
Deep Learning Model - Root Mean Squared Error (RMSE): 99.14609
Deep Learning Model - R-squared: 0.47151264453397124


In [ ]:
# # Export to pickle file in Google Drive to continue in VSCode
# import pickle

# with open('/content/drive/My Drive/EL_machine_learning_model.pkl', 'wb') as model_file:
#     pickle.dump(nn, model_file)

In [ ]:
# Stop the Spark session when you're done
spark.stop()